In [1]:
import os
import cvxopt
import numpy as np
import scipy.special
import matplotlib.pyplot as plt 

from cvxopt import solvers
from scipy.linalg import block_diag

In [2]:
def bernstein_coeff_order10_new(n, tmin, tmax, t_actual):
    l = tmax - tmin
    t = (t_actual - tmin) / l
    
    P0 = scipy.special.binom(n, 0) * ((1 - t) ** (n - 0)) * t ** 0
    P1 = scipy.special.binom(n, 1) * ((1 - t) ** (n - 1)) * t ** 1
    P2 = scipy.special.binom(n, 2) * ((1 - t) ** (n - 2)) * t ** 2
    P3 = scipy.special.binom(n, 3) * ((1 - t) ** (n - 3)) * t ** 3
    P4 = scipy.special.binom(n, 4) * ((1 - t) ** (n - 4)) * t ** 4
    P5 = scipy.special.binom(n, 5) * ((1 - t) ** (n - 5)) * t ** 5
    P6 = scipy.special.binom(n, 6) * ((1 - t) ** (n - 6)) * t ** 6
    P7 = scipy.special.binom(n, 7) * ((1 - t) ** (n - 7)) * t ** 7
    P8 = scipy.special.binom(n, 8) * ((1 - t) ** (n - 8)) * t ** 8
    P9 = scipy.special.binom(n, 9) * ((1 - t) ** (n - 9)) * t ** 9
    P10 = scipy.special.binom(n, 10) * ((1 - t) ** (n - 10)) * t ** 10

    P0dot = -10.0 * (-t + 1) ** 9
    P1dot = -90.0 * t * (-t + 1) ** 8 + 10.0 * (-t + 1) ** 9
    P2dot = -360.0 * t ** 2 * (-t + 1) ** 7 + 90.0 * t * (-t + 1) ** 8
    P3dot = -840.0 * t ** 3 * (-t + 1) ** 6 + 360.0 * t ** 2 * (-t + 1) ** 7
    P4dot = -1260.0 * t ** 4 * (-t + 1) ** 5 + 840.0 * t ** 3 * (-t + 1) ** 6
    P5dot = -1260.0 * t ** 5 * (-t + 1) ** 4 + 1260.0 * t ** 4 * (-t + 1) ** 5
    P6dot = -840.0 * t ** 6 * (-t + 1) ** 3 + 1260.0 * t ** 5 * (-t + 1) ** 4
    P7dot = -360.0 * t ** 7 * (-t + 1) ** 2 + 840.0 * t ** 6 * (-t + 1) ** 3
    P8dot = 45.0 * t ** 8 * (2 * t - 2) + 360.0 * t ** 7 * (-t + 1) ** 2
    P9dot = -10.0 * t ** 9 + 9 * t ** 8 * (-10.0 * t + 10.0)
    P10dot = 10.0 * t ** 9

    P0ddot = 90.0 * (-t + 1) ** 8
    P1ddot = 720.0 * t * (-t + 1) ** 7 - 180.0 * (-t + 1) ** 8
    P2ddot = 2520.0 * t ** 2 * (-t + 1) ** 6 - 1440.0 * t * (-t + 1) ** 7 + 90.0 * (-t + 1) ** 8
    P3ddot = 5040.0 * t ** 3 * (-t + 1) ** 5 - 5040.0 * t ** 2 * (-t + 1) ** 6 + 720.0 * t * (-t + 1) ** 7
    P4ddot = 6300.0 * t ** 4 * (-t + 1) ** 4 - 10080.0 * t ** 3 * (-t + 1) ** 5 + 2520.0 * t ** 2 * (-t + 1) ** 6
    P5ddot = 5040.0 * t ** 5 * (-t + 1) ** 3 - 12600.0 * t ** 4 * (-t + 1) ** 4 + 5040.0 * t ** 3 * (-t + 1) ** 5
    P6ddot = 2520.0 * t ** 6 * (-t + 1) ** 2 - 10080.0 * t ** 5 * (-t + 1) ** 3 + 6300.0 * t ** 4 * (-t + 1) ** 4
    P7ddot = -360.0 * t ** 7 * (2 * t - 2) - 5040.0 * t ** 6 * (-t + 1) ** 2 + 5040.0 * t ** 5 * (-t + 1) ** 3
    P8ddot = 90.0 * t ** 8 + 720.0 * t ** 7 * (2 * t - 2) + 2520.0 * t ** 6 * (-t + 1) ** 2
    P9ddot = -180.0 * t ** 8 + 72 * t ** 7 * (-10.0 * t + 10.0)
    P10ddot = 90.0 * t ** 8

    P = np.hstack((P0, P1, P2, P3, P4, P5, P6, P7, P8, P9, P10))
    Pdot = np.hstack((P0dot, P1dot, P2dot, P3dot, P4dot, P5dot, P6dot, P7dot, P8dot, P9dot, P10dot)) / l
    Pddot = np.hstack((P0ddot, P1ddot, P2ddot, P3ddot, P4ddot, P5ddot, P6ddot, P7ddot, P8ddot, P9ddot, P10ddot)) / (l ** 2)
    return P, Pdot, Pddot

#### Initializations

In [3]:
x_min = -6.0
x_max = 6.0
y_min = -6.0
y_max = 6.0

a_obs = 1.0
b_obs = 1.0

# t_fin = 6.0
# num = 75
t_fin = 4.0
num = 50

In [4]:
tot_time = np.linspace(0.0, t_fin, num)
tot_time_copy = tot_time.reshape(num, 1)
P, Pdot, Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)
nvar = np.shape(P)[1]
num = np.shape(P)[0]

In [5]:
# x_obs_temp = np.hstack((-2.0, -0.79, 3.0))
# y_obs_temp = np.hstack((-2.0, 1.0, -0.80))
x_obs_temp = np.hstack((-2.0, -0.79, 3.0, 4.0))
y_obs_temp = np.hstack((-2.0, 1.0, -0.80, 2.0))
num_obs = np.shape(x_obs_temp)[0]

x_obs = np.ones((num_obs, num)) * x_obs_temp[:, np.newaxis]
y_obs = np.ones((num_obs, num)) * y_obs_temp[:, np.newaxis]
obs_pos = np.vstack((x_obs_temp, y_obs_temp)).T

In [6]:
A_eq = np.vstack((P[0], Pdot[0], Pddot[0], P[-1], Pdot[-1], Pddot[-1]))
A_obs = np.tile(P, (num_obs, 1))
Q_smoothness = np.dot(Pddot.T, Pddot)

In [7]:
def is_obstacle_collision(pos):
    # pos: 1 x 2, obs_pos: n x 2
    dis = np.linalg.norm(pos - obs_pos, axis=1)
    return np.sum(dis > 1.5) < num_obs

In [8]:
def is_goal_collision(init_pos, goal_pos):
    dis = np.linalg.norm(goal_pos - init_pos, axis=1)
    return np.sum(dis > 1.5) < 1

In [9]:
def generate_init_pos():
    while True:
        init_pos = np.random.uniform(x_min + 0.5, x_max - 0.5, (1, 2))
        if not is_obstacle_collision(init_pos):
            break
    return init_pos

In [10]:
def generate_goal_pos(init_pos):
    while True:
        goal_pos = np.random.uniform(x_min + 0.5, x_max - 0.5, (1, 2))
        if not is_obstacle_collision(goal_pos) and not is_goal_collision(init_pos, goal_pos):
            break
    return goal_pos

In [11]:
def initialize():
    init_pos = generate_init_pos()
    goal_pos = generate_goal_pos(init_pos)
    
    x_init, y_init = init_pos.reshape(-1)
    x_fin, y_fin = goal_pos.reshape(-1)
    
    vx_init, vy_init = np.random.uniform(0, 1, 2)
    ax_init, ay_init = np.random.uniform(0, 0.5, 2)
    
    vx_fin, vy_fin = np.random.uniform(0, 1, 2)
    ax_fin, ay_fin = np.random.uniform(0, 0.5, 2)
    
#     vx_fin, vy_fin = 0.0, 0.0
#     ax_fin, ay_fin = 0.0, 0.0
    
    return x_init, y_init, vx_init, vy_init, ax_init, ay_init, x_fin, y_fin, vx_fin, vy_fin, ax_fin, ay_fin

#### Weight Initializations

In [12]:
# rho_obs = 0.3
rho_obs = 1.2
rho_eq = 10.0
weight_smoothness = 10

In [13]:
def compute_sol(rho_obs, rho_eq, weight_smoothness, num_obs, bx_eq, by_eq, P, Pdot, Pddot, x_obs, y_obs, a_obs, b_obs):    
    maxiter = 300
    nvar = np.shape(P)[1]
    num = np.shape(P)[0]

    A_eq = np.vstack((P[0], Pdot[0], Pddot[0], P[-1], Pdot[-1], Pddot[-1]))
    A_obs = np.tile(P, (num_obs, 1))

    cost_smoothness = weight_smoothness * np.dot(Pddot.T, Pddot)

    alpha_obs = np.zeros((num_obs, num))
    d_obs = np.ones((num_obs, num))

    lamda_x = np.zeros(nvar)
    lamda_y = np.zeros(nvar)
    res_obs = np.ones(maxiter)
    res_eq = np.ones(maxiter)
    d_min = np.ones(maxiter)
    cost = cost_smoothness + rho_obs * np.dot(A_obs.T, A_obs) + rho_eq * np.dot(A_eq.T, A_eq)
    
    for i in range(0, maxiter):
        temp_x_obs = d_obs * np.cos(alpha_obs) * a_obs
        b_obs_x = x_obs.reshape(num * num_obs) + temp_x_obs.reshape(num * num_obs)

        temp_y_obs = d_obs * np.sin(alpha_obs) * b_obs
        b_obs_y = y_obs.reshape(num * num_obs) + temp_y_obs.reshape(num*num_obs)

        lincost_x = -lamda_x - rho_obs * np.dot(A_obs.T, b_obs_x) - rho_eq * np.dot(A_eq.T, bx_eq)
        lincost_y = -lamda_y - rho_obs * np.dot(A_obs.T, b_obs_y) - rho_eq * np.dot(A_eq.T, by_eq)

        sol_x = np.linalg.solve(-cost, lincost_x)
        sol_y = np.linalg.solve(-cost, lincost_y)

        x = np.dot(P, sol_x)
        y = np.dot(P, sol_y)

        wc_alpha = (x - x_obs)
        ws_alpha = (y - y_obs)
        alpha_obs = np.arctan2(ws_alpha * a_obs, wc_alpha * b_obs)
        
        c1_d = 1.0 * rho_obs * (a_obs ** 2 * np.cos(alpha_obs) ** 2 + b_obs ** 2 * np.sin(alpha_obs) ** 2)
        c2_d = 1.0 * rho_obs * (a_obs * wc_alpha * np.cos(alpha_obs) + b_obs * ws_alpha * np.sin(alpha_obs))

        d_temp = c2_d / c1_d
        d_obs = np.maximum(np.ones((num_obs, num)), d_temp)
        d_min[i] = np.amin(d_temp)

        res_x_obs_vec = wc_alpha - a_obs * d_obs * np.cos(alpha_obs)
        res_y_obs_vec = ws_alpha - b_obs * d_obs * np.sin(alpha_obs)
        
        res_eq_x_vec = np.dot(A_eq, sol_x) - bx_eq
        res_eq_y_vec = np.dot(A_eq, sol_y) - by_eq

        lamda_x = lamda_x-rho_obs*np.dot(A_obs.T, res_x_obs_vec.reshape(num_obs * num)) - rho_eq * np.dot(A_eq.T, res_eq_x_vec)
        lamda_y = lamda_y-rho_obs*np.dot(A_obs.T, res_y_obs_vec.reshape(num_obs * num)) - rho_eq * np.dot(A_eq.T, res_eq_y_vec)

        res_eq[i] = np.linalg.norm(np.hstack((res_eq_x_vec,  res_eq_y_vec)))
        res_obs[i] = np.linalg.norm(np.hstack((res_x_obs_vec, res_y_obs_vec)))

    slack_obs = np.sqrt((d_obs - 1))
    return x, y, sol_x, sol_y, alpha_obs.reshape(num_obs*num), d_obs.reshape(num_obs*num), lamda_x, lamda_y, slack_obs.reshape(num_obs*num)

In [14]:
os.makedirs('./multiple_loss_test/plots', exist_ok=True)
os.makedirs('./multiple_loss_test/data', exist_ok=True)

In [15]:
def save_traj_figs(x_init, y_init, x_fin, y_fin, x_traj, y_traj, traj_num):
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.scatter(x_init, y_init, label='Initial Pos')
    ax.scatter(x_fin, y_fin, label='Final Pos')
    ax.plot(x_traj, y_traj, '-b', linewidth=3.0)
    
    th = np.linspace(0, 2 * np.pi, 100)
    for i in range(0, num_obs):
        x_circ = x_obs_temp[i] + a_obs * np.cos(th)
        y_circ = y_obs_temp[i] + b_obs * np.sin(th)
        ax.plot(x_circ, y_circ, '-k')
    
    ax.legend()
    ax.set_xlim([-7, 7])
    ax.set_ylim([-7, 7])
    plt.savefig('./multiple_loss_test/plots/{}.png'.format(traj_num))
    plt.close()

In [16]:
def generate_trajectories():
    x_init, y_init, vx_init, vy_init, ax_init, ay_init, x_fin, y_fin, vx_fin, vy_fin, ax_fin, ay_fin = initialize()

    bx_eq =  np.hstack((x_init, vx_init, ax_init, x_fin, vx_fin, ax_fin))
    by_eq =  np.hstack((y_init, vy_init, ay_init, y_fin, vy_fin, ay_fin))
    
    x, y, sol_x, sol_y, alpha_obs, d_obs, lamda_x, lamda_y, slack_obs = compute_sol(rho_obs, rho_eq, weight_smoothness, num_obs, bx_eq, by_eq, P, Pdot, Pddot, x_obs, y_obs, a_obs, b_obs)
    return x, y, x_init, y_init, vx_init, vy_init, ax_init, ay_init, x_fin, y_fin, vx_fin, ax_fin, vy_fin, ay_fin

In [17]:
for i in range(1000):
    x_traj, y_traj, x_init, y_init, vx_init, vy_init, ax_init, ay_init, x_fin, y_fin, vx_fin, ax_fin, vy_fin, ay_fin = generate_trajectories()
    save_traj_figs(x_init, y_init, x_fin, y_fin, x_traj, y_traj, i)
    np.save("./multiple_loss_test/data/{}.npy".format(i), {
        "x_traj": x_traj,
        "y_traj": y_traj,
        "x_init": x_init,
        "y_init": y_init,
        "vx_init": vx_init,
        "vy_init": vy_init,
        "ax_init": ax_init,
        "ay_init": ay_init,
        "x_fin": x_fin,
        "y_fin": y_fin,
        "vx_fin": vx_fin,
        "ax_fin": ax_fin,
        "vy_fin": vy_fin,
        "ay_fin": ay_fin,
    })